# BIP's submission function

A function which goal is to return a dataframe with the right data structure to be delivered. It starts from a dataframe with predictions.

The input dataframe should **at least** contain:

| StoreID  | D_Month | _NumberOfSales |
| ---------|---------| ---------------|
| 1000     | 3       | 16             |
| 1000     | 4       | 23             |
| 1001     | 3       | 411            |
| 1001     | 4       | 3120           |
| 1002     | 3       | 8              |

Where: 
 - *_NumberOfSales* are **the predicted values**
 
 
### NB: No check is done on the months. If they're not 3, 4, they're left "AS IS"
 

Start from test set to simulate a predicted dataset

In [1]:
from import_man import *

sub_features = ['StoreID', 'D_Month', '_NumberOfSales']

df = pd.read_csv('./dataset/RFR5_fake_test.csv')

df.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,NearestCompetitor,Region,NumberOfSales,Region_AreaKM2,Region_GDP,...,t-6,t-15,t-21,t-13,t-20,MeanStoreSales,MeanRegionSales,D_Day_of_week,_NumberOfSales,sqrd_diff
0,1000,2018-01-01,0,1,0,326,7,8540.000000,9643,17130,...,10650.000000,7675.446488,7368.0,10520.0,6858.0,7682.751625,4698.572717,0,9776.962748,2.503328e+07
1,1000,2018-01-02,0,1,0,326,7,10364.000000,9643,17130,...,4498.000000,12353.000000,6858.0,9479.0,5890.0,7682.751625,4698.572717,1,6018.866667,1.550600e+06
2,1000,2018-01-03,0,1,0,326,7,4676.000000,9643,17130,...,7675.446488,10520.000000,5890.0,10529.0,6913.0,7682.751625,4698.572717,2,6234.658815,2.134588e+06
3,1000,2018-01-04,1,0,0,326,7,7675.446488,9643,17130,...,7675.446488,9479.000000,6913.0,9050.0,7282.0,7682.751625,4698.572717,3,6513.012222,3.025430e+06
4,1000,2018-01-05,0,1,0,326,7,6267.000000,9643,17130,...,8930.000000,10529.000000,7282.0,9835.0,9179.0,7682.751625,4698.572717,4,7242.277146,6.094190e+06


In [2]:
df = df[sub_features]
df.head()

,StoreID,D_Month,_NumberOfSales
0,1000,1,9776.962748
1,1000,1,6018.866667
2,1000,1,6234.658815
3,1000,1,6513.012222
4,1000,1,7242.277146


In [6]:
# let's sum NumberOfSales by the store and month
df = df.groupby(['StoreID', 'D_Month']).sum()

In [7]:
# name conversion from our standard to BIP submission requirements
df.rename(index=str, columns={"D_Month": "Month", "_NumberOfSales": "NumberOfSales"}, inplace=True)
df.head()

NumberOfSales
StoreID D_Month               
1000    1        242944.048532
        2        237634.506868
1001    1        108036.745647
        2         97937.201092
1002    1        157617.417976

In [5]:
df.to_csv('./dataset/submission_fake.csv')